# Imports

In [2]:
%reload_ext autoreload
%autoreload 2

import requests



import warnings
warnings.filterwarnings("ignore")
# import warnings

# def fxn():
#     warnings.warn("deprecated", DeprecationWarning)

# with warnings.catch_warnings():
#     warnings.simplefilter("ignore")
#     fxn()

import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.utils import formataddr

import pandas as pd
import time, requests, csv, json
from datetime import datetime
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 20000)
pd.set_option('display.max_colwidth', -1)

import sys, os, shutil, pdb
sys.path.insert(1, '/Users/muhammadfaisal/Documents/')


from pathlib import Path
import urllib.request

import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from IPython.display import display, clear_output
from bs4 import BeautifulSoup




from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

try: # running remotely
    from google.colab import drive 
except: # running locally
    from utils import *
    from engine import GDrive


In [19]:
# %pip install tabula-py

# Local VS Remote

In [20]:
def download_scripts():
    # downloading utils and engine scripts from github
    r=requests.get("https://raw.githubusercontent.com/faysal887/tricyons-analyser-v2/master/engine.py")
    with open('engine.py', 'w') as f: f.write(r.content.decode("utf-8"))

    r=requests.get("https://raw.githubusercontent.com/faysal887/tricyons-analyser-v2/master/utils.py")
    with open('utils.py', 'w') as f: f.write(r.content.decode("utf-8"))

In [21]:
'''
checking whether the code is running on local machine or on collab
'''

LOCAL=None # running code locally or remotely
try:
    drive.mount('/content/drive')

    GoogleAuth.DEFAULT_SETTINGS['client_config_file'] = '/content/drive/MyDrive/Colab Notebooks/client_secrets.json'
    gauth = GoogleAuth()
    gauth.GetAuthUrl()
    print('running on collab')
    LOCAL=False
except Exception as e:
    print(e)
    print('running locally')
    LOCAL=True


if LOCAL: # running locally
    try:
        if g: print('GDrive already connected!')
    except: 
        g=GDrive()
        print('created new connection!')
else: # running remotely
    drive.mount('/content/drive')

    GoogleAuth.DEFAULT_SETTINGS['client_config_file'] = '/content/drive/MyDrive/Colab Notebooks/client_secrets.json'
    gauth = GoogleAuth()
    print(gauth.GetAuthUrl())
    redirect_url=input('Enter redirect url')
    code=redirect_url.split('code=')[1].split('&scope=')[0]
    gauth.Authenticate(code)
    download_scripts()
    from utils import *
    from engine import GDrive
    g=GDrive(gauth)



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
running on collab
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
https://accounts.google.com/o/oauth2/auth?client_id=454591216913-28jl16cqrsojhifrtmopf0ts9mb4281t.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code
Enter redirect urlhttp://localhost/?code=4/0AVHEtk6x12k5OcZkqF8pe0fUVUnpzR0YCLLI1sqImPERYA5FTNZ8p11u_qtHggYJzKPpqA&scope=https://www.googleapis.com/auth/drive
Authentication successful.


# Download Catalogs

In [22]:
TMP_DIR='./tmp'
try: shutil.rmtree(TMP_DIR)
except: pass

In [23]:
# main

tmp_dir='./tmp'
merged_dir=f'{tmp_dir}/merged'
online_catalogs=f'{tmp_dir}/online_catalogs'
gdrive_catalogs=f'{tmp_dir}/gdrive_catalogs'

# 1. create /tmp if not
Path(tmp_dir).mkdir(parents=True, exist_ok=True)
Path(merged_dir).mkdir(parents=True, exist_ok=True)
Path(online_catalogs).mkdir(parents=True, exist_ok=True)
Path(gdrive_catalogs).mkdir(parents=True, exist_ok=True)
print('done 1')

# 2. download excel sheet for links and emails 
tree=['Supplier Catalogues', 'AUTOMATION', 'Brands For Auto-Email']
folder_id=g.find_gdrive_folder_id(tree=tree, folder_id='root')
g.download_files(folder_id, tmp_dir)

linksdf=pd.read_excel(f'{tmp_dir}/Catalogs Link.xlsx')
# data=linksdf.dropna(subset=['Link'], axis=0)
data=strip_column_names(linksdf)
print(data.shape)
print('done 2')



done 1
(41, 16)
done 2


In [24]:
# 3. download excel catalogs
df=data[(data.Active==True) & (data.Type=='LINK')]
print(df.shape)
catalogs=download_online_excel_catalogs(df, online_catalogs, test_catalogs=[])
print('done 3') 

(24, 16)
Invalid URL '/export?format=csv': No scheme supplied. Perhaps you meant http:///export?format=csv?

 ************************************************* 

Error Catalogs:  {'epilsonwholesale': Exception('Catalog Empty'), 'kntradingllc': Exception('Catalog Empty'), 'ecomwholesaledeals': Exception('Catalog Empty')}

 ************************************************* 

done 3


In [ ]:
# 4. delete existing supplier folders
tree_base=['Supplier Catalogues', 'AUTOMATION', 'Catalogs_ALL']
df.Distributor.apply(lambda x: g.delete_by_name(tree_base+[x]))
print('done 4')

done 4


In [ ]:
# 5. create new supplier folders
tree=['Supplier Catalogues', 'AUTOMATION', 'Catalogs_ALL']
parent_folder_id=g.find_gdrive_folder_id(tree=tree, folder_id='root')
df['folder_id'] = df.Distributor.apply(lambda x: g.create_folder(parent_folder_id, x))
print('done 5')


# 6. create folder for catalog in supplier folder
df['catalog_folder_id'] = df.folder_id.apply(lambda x: g.create_folder(parent_folder_id=x, folder_name='catalog'))
print('done 6')



done 5
done 6


In [ ]:
# 7. upload catalog
# _ = df.apply(lambda x: g.upload_file(folder_id=x.catalog_folder_id, file_path=f'{online_catalogs}/{x.Distributor}.xlsx'), axis=1)
for i, row in df.iterrows():
    print(f'{i+1}/{len(df)}', end='\r')
    file_path=f'{online_catalogs}/{row.Distributor}.xlsx'
    if os.path.exists(file_path):
        g.upload_file(folder_id=row.catalog_folder_id, file_path=file_path)
    
print('done 7')


# 8. delete /tmp
# shutil.rmtree(tmp_dir)
# print('done 8')

done 7


In [25]:
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
                                # PART-2: Start again with data #
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
df=data[data.Active==True].copy()
print(df.shape)

(31, 16)


In [26]:


# 8. for all the suppliers in catalogs_all, get their catalog folder_ids 
for i, row in df.iterrows():
    try:
        print(f'{i+1}/{len(df)}', end='\r')
        tree=['Supplier Catalogues', 'AUTOMATION', 'Catalogs_ALL', row.Distributor, 'catalog']
        catalog_folder_id=g.find_gdrive_folder_id(tree=tree, folder_id='root')
        df.loc[i, 'catalog_folder_id']=catalog_folder_id
    except Exception as e:
        print('Error: ', row['Distributor'], e)
print('done 8')



Error:  EpilsonWholesale can only convert an array of size 1 to a Python scalar
Error:  GSCommodityTrading can only convert an array of size 1 to a Python scalar
Error:  MinMaxDeals can only convert an array of size 1 to a Python scalar
Error:  KNTradingLLC can only convert an array of size 1 to a Python scalar
Error:  EcomWholesaleDeals can only convert an array of size 1 to a Python scalar
done 8


In [27]:
# 9. loop on supplier folders id df and download all catalogs from gdrive
# _ = df.catalog_folder_id.apply(lambda x: g.download_files(gdrive_folder_id=x, local_folder_path=gdrive_catalogs))

for i, row in df[['Distributor','catalog_folder_id']].iterrows():
    print(f'{i+1}/{len(df)}', end='\r')

    try:
        dist, _id=row['Distributor'], row['catalog_folder_id']
        g.download_files(gdrive_folder_id=_id, local_folder_path=gdrive_catalogs)
    except Exception as e:
        print('Error: ', dist, e)
print('done 9')

Error:  DPCIWholesale No downloadLink/exportLinks for mimetype found in metadata
Error:  EpilsonWholesale <HttpError 404 when requesting https://www.googleapis.com/drive/v2/files?q=%27nan%27+in+parents+and+trashed%3Dfalse&maxResults=1000&alt=json returned "File not found:". Details: "[{'message': 'File not found: ', 'domain': 'global', 'reason': 'notFound', 'location': 'file', 'locationType': 'other'}]">
Error:  GSCommodityTrading <HttpError 404 when requesting https://www.googleapis.com/drive/v2/files?q=%27nan%27+in+parents+and+trashed%3Dfalse&maxResults=1000&alt=json returned "File not found:". Details: "[{'message': 'File not found: ', 'domain': 'global', 'reason': 'notFound', 'location': 'file', 'locationType': 'other'}]">
Error:  MinMaxDeals <HttpError 404 when requesting https://www.googleapis.com/drive/v2/files?q=%27nan%27+in+parents+and+trashed%3Dfalse&maxResults=1000&alt=json returned "File not found:". Details: "[{'message': 'File not found: ', 'domain': 'global', 'reason': '

In [28]:
# 10. create new catalogs dict obj

catalogs={}
for i, fn in enumerate(os.listdir(gdrive_catalogs)):
    print(f'{i+1}/{len(os.listdir(gdrive_catalogs))}', end='\r')

    if '.xlsx' in fn or 'xls' in fn: tmpdf=pd.read_excel(f'{gdrive_catalogs}/{fn}')
    if '.csv'   in fn: tmpdf=pd.read_csv(f'{gdrive_catalogs}/{fn}')

    fn=fn.split('.')[0]

    catalogs[fn]=tmpdf
    # catalogs[fn.replace('.xlsx','').replace('.csv','').replace('.csv','')]=tmpdf

print('done 10')


done 10


In [16]:
# sanity checks to check all the required columns are present
for name, tmpdf in catalogs.items():
    # pdb.set_trace()
    id_columns_name	= df[df.Distributor==name].id_columns_name.item()
    id_columns_type = df[df.Distributor==name].id_columns_type.item()
    price_column    = df[df.Distributor==name].price_column.item()

    if id_columns_name not in tmpdf.columns: print(f'{name} {id_columns_name} column missing')
    if price_column not in tmpdf.columns: print(f'{name} {price_column} column missing')

BGSales price column missing
LvDistribution ASIN column missing
LvDistribution Customer MOQ Cost column missing


In [17]:
# 11. label data
df, catalogs=label_data(df, catalogs)
print('done 11')

These suppliers are unlabelled:  ['NoveltyIncWholesale']
done 11


In [32]:
# preprocess each supplier separately
for name, tmpdf in catalogs.items():
    if name == 'NUK':                tmpdf=preprocess_nuk(tmpdf)
    if name == 'BGSales':            tmpdf=preprocess_bgsales(tmpdf)
    if name == 'EcomWholesaleDeals': tmpdf=preprocess_ewd(tmpdf)

    catalogs[name]=tmpdf

NameError: ignored

In [ ]:
# label encode supplier name in df dataframe
# keys=df.Distributor.unique().tolist()
# values=['TRI'+str(i).zfill(5) for i in range(len(keys))]
# labels_map=dict(zip(keys, values))
# df['distributor_key'] = df.Distributor.map(labels_map)

# now this is being done manually
# print('done 12')


# apply labels_map for each catalog in catalog dict of dfs
for key, tmpdf in catalogs.items():
    tmpdf['distributor_key']=df[df.Distributor==key].distributor_key.item()
    catalogs[key]=tmpdf


# 12. upload updated catalog_links df
# df.drop(['folder_id', 'catalog_folder_id'], axis=1).to_excel(f'{tmp_dir}/Catalogs Link.xlsx', index=False)

In [ ]:
# 13. check for missing values in each column
missing_total=0
for name, tmpdf in catalogs.items():
    # print(name, tmpdf['each_price'].isnull().sum())
    missing_total+=tmpdf['each_price'].isnull().sum()
print(missing_total)
print('done 12')

19
done 12


In [ ]:
# 13. preprocess asins
asins_suppliers=df[df.id_columns_type=='asin'].Distributor.tolist()

tmpli=[]
for x in asins_suppliers:
  try:
    tmpli.append(catalogs[x][['asin', 'each_price', 'distributor_key']])
  except Exception as e:
    print(f'Error in {x}: e')
asinsdf = pd.concat(tmpli)
print('after concating all asinsdf: ',asinsdf.shape, asinsdf.each_price.isnull().sum())

asinsdf=asinsdf.dropna()
print('after dropna: ',asinsdf.shape, asinsdf.each_price.isnull().sum())

asinsdf=preprocess_price(asinsdf)
print('after preprocess_price: ',asinsdf.shape, asinsdf.each_price.isnull().sum())

asinsdf=asinsdf.reset_index(drop=True)
print('after reset_index: ',asinsdf.shape, asinsdf.each_price.isnull().sum())

asinsdf.to_excel(f'{merged_dir}/asinsdf.xlsx', index=False)
print('after saving: ', asinsdf.shape)
print('done 13')

after concating all asinsdf:  (11442, 3) 17
after dropna:  (11381, 3) 0
after preprocess_price:  (11381, 3) 0
after reset_index:  (11381, 3) 0
after saving:  (11381, 3)
done 13


In [ ]:
# 14. preprocess upcs
upc_suppliers=df[df.id_columns_type=='upc'].Distributor.tolist()
# upcsdf = pd.concat([catalogs[x][['upc', 'each_price', 'distributor_key']] for x in upc_suppliers])
tmpli=[]
for x in upc_suppliers:
  try:
    tmpli.append(catalogs[x][['upc', 'each_price', 'distributor_key']])
  except Exception as e:
    print(f'Error in {x}: {e}')
upcsdf = pd.concat(tmpli)
print('after concat: ',upcsdf.shape, upcsdf.each_price.isnull().sum())

upcsdf=upcsdf.dropna(subset=['upc', 'each_price'])
print('after dropna: ',upcsdf.shape, upcsdf.each_price.isnull().sum())

upcsdf=preprocess_upc(upcsdf)
print('after preprocess_upc: ',upcsdf.shape, upcsdf.each_price.isnull().sum())

upcsdf=preprocess_price(upcsdf)
print('after preprocess_price: ',upcsdf.shape, upcsdf.each_price.isnull().sum())

upcsdf=upcsdf.reset_index(drop=True)
print('after reset_index: ',upcsdf.shape, upcsdf.each_price.isnull().sum())

upcsdf.to_excel(f'{merged_dir}/upcsdf.xlsx', index=False)
print('after save: ',upcsdf.shape, upcsdf.each_price.isnull().sum())

print('done 14')

Error in DPCIWholesale: 'DPCIWholesale'
after concat:  (93220, 3) 2
after dropna:  (92046, 3) 0
after preprocess_upc:  (92034, 3) 0
after preprocess_price:  (92034, 3) 0
after reset_index:  (92034, 3) 0
after save:  (92034, 3) 0
done 14


In [ ]:
# 15. upload merged asin and upc on gdrive
tree=['Supplier Catalogues', 'AUTOMATION', 'ScanUnlimited_ALL']
folder_id=g.find_gdrive_folder_id(tree, folder_id='root')
# delete existing ScanUnlimited_ALL
g.delete_by_id(folder_id)
# create new ScanUnlimited_ALL
tree=['Supplier Catalogues', 'AUTOMATION']
parent_folder_id=g.find_gdrive_folder_id(tree, folder_id='root')
new_folder_id=g.create_folder(parent_folder_id, 'ScanUnlimited_ALL')
g.upload_file(new_folder_id, f'{merged_dir}/asinsdf.xlsx')
g.upload_file(new_folder_id, f'{merged_dir}/upcsdf.xlsx')
print('done 16')

done 16


In [ ]:
print(f'''
    Total:   {asinsdf.shape[0]+upcsdf.shape[0]}
    upcsdf:  {upcsdf.shape[0]}
    asinsdf: {asinsdf.shape[0]}
''')


    Total:   103415
    upcsdf:  92034
    asinsdf: 11381



In [ ]:
try: shutil.rmtree(TMP_DIR)
except: pass

# TODO

In [ ]:
# add sheet name column so it shortens the download function
# change ignore to active
# add other columns list column

# Send Emails

In [ ]:
def send_email(addr, subject, body):
    fromaddr = "contact@tricyons.com"
    pswd     = "tricyons123"

    msg = MIMEMultipart()
    msg['From'] = formataddr(('Tricyons LLC', fromaddr))
    msg['To'] = addr
    msg['Subject'] = subject
    body = body

    msg.attach(MIMEText(body, 'plain'))
    # smtpserver = smtplib.SMTP("mail.tricyons.com", 465) # tls

    server = smtplib.SMTP("mail.tricyons.com", 26) # non-tls
    server.starttls()
    server.login(fromaddr, pswd)

    text = msg.as_string()
    server.sendmail(fromaddr, addr, text)
    server.quit()    
    return True

### Ask Catalog

In [ ]:
emailsdf=data[(data.Active==True) & (data.Type=='EMAIL')].sort_values('Email', ascending=False)
print(emailsdf.shape)
emailsdf

In [ ]:
with open('./email_catalog.txt', 'r') as f:
    email_body_template = f.read()

print(email_body_template)

In [ ]:
email_subject='Updated Catalog'

for i, row in emailsdf.iterrows():
    print(f'{i+1}/{emailsdf.shape[0]}', end='\r')

    email_to=row.Email.strip().lower()

    email_body=email_body_template.replace('SUBJECT_NAME_TAG', row.Email_Subject)

    email_sent = send_email(email_to, email_subject, email_body)
    if not email_sent: print(f'Error in sending email to: {row.Email_Subject}')

    time.sleep(3)

### Account Open

In [ ]:


with open('./email_template.txt', 'r') as f:
    email_body_template = f.read()

print(email_body_template)


# email_to='faysalaslam887@gmail.com'
# email_subject = "Testing Email Body 3 Spaces"
# email_body=email_body_template.replace('COMPANY_NAME_TAG', 'Test Company')
# send_email(email_to, email_subject, email_body)

# g=GDrive()


'''
tmp_dir='./tmp/'

tree=['Supplier Catalogues', 'Brands For Auto-Email']
g.supplier_id=g.find_gdrive_folder_id(tree=tree, folder_id='root')
g.download_files(g.supplier_id, tmp_dir)

all_suppliers=pd.read_excel(f'{tmp_dir}/All Suppliers.xlsx')
print(all_suppliers.shape)

all_suppliers.head()

df=all_suppliers.dropna(subset=['Email'], axis=0).reset_index(drop=True)
print(df.shape)
df.head()

# email_subject = "Wholesale Account Opening"

# for i, row in df.iloc[78:].iterrows():
#     print(f'{i+1}/{df.shape[0]}', end='\r')

#     email_to=row.Email.strip().lower()

#     supplier_name = row.Name.strip()

#     email_body=email_body_template.replace('COMPANY_NAME_TAG', supplier_name)

#     email_sent = send_email(email_to, email_subject, email_body)
#     if not email_sent: print(f'Error in sending email to: {supplier_name}')

#     time.sleep(3)
    
'''

# Test New Catalogs

In [3]:
from utils import *

In [ ]:
# EpilsonWholesale
df2=get_catalog_google_sheets_2(url, sheet_name='Sheet1')
df2=df2.iloc[1:]
df2=make_first_row_header(df2)
df2=df2[~df2.ASIN.astype(str).str.contains('--')]
df2=df2.dropna()

In [ ]:
# GSCommodityTrading
df1=get_catalog_google_sheets(url).reset_index(drop=True)
print(df1.shape)
df1=df1.iloc[9:]
df1.head(20)


In [ ]:
# MinMaxDeals
df1=get_catalog_google_sheets(url).reset_index(drop=True)
print(df1.shape)

In [ ]:
# KNTradingLLC
df2=get_catalog_google_sheets_2(url, sheet_name='Sheet1')
df2=make_first_row_header(df2)
df2=df2[[x for x in df2.columns.tolist() if x!=None]]


In [114]:
# EcomWholesaleDeals 



(82, 4)
(165, 4)
(109, 4)


,DESCRIPTION,UPC,LINK,COST
0,AQUAFRESH TOOTHPASTE TRIPLE PROTECT PUMP,5000347027147,Amazon,$2.00
1,B.TAN SELF TAN MOUSSE DARK FAKE IT TILL MAKE IT,9347108008721,Amazon,$2.10
2,BIORE 200ML BLUE AGAVE & BAKING SODA BALANCING PORE CLEANSER,5017634258191,Amazon,$2.50
3,"Brut Original Edt Spray for Men, 3.38 Ounce",3014230021039,Amazon,$4.25
4,"Brylcreem Hairdressing Original Gel, 250 ml",5000231039546,Amazon,$3.00
5,Bull Dog Skincare for Men Original Shower Gel 6.70 Ounces,NaN,Amazon,$2.50
6,BULLDOG MOISTURISER ORIGINAL 100ML,5060144648181,Amazon,$4.25
7,"Bulldog Natural Skincare Original Face Wash For Men, 5 Oz",5060144642240,Amazon,$2.50
8,CLEAN & CLEAR 15ML GEL ADVANTAGE QUICK CLEAR SPOT TREATMENT,3574660178821,Amazon,$12.50
9,CLEARASIL RAPID ACTION TREAT CREAM,5011417544846,Amazon,$4.50


In [87]:
links={
    'EpilsonWholesale': "https://img1.wsimg.com/blobby/go/271277f5-5f94-472c-b153-578386146f02/downloads/COSMETIC%20SPREED%20SHEET%20%20(1).xlsx?ver=1680857754932",
    'GSCommodityTrading': "https://docs.google.com/spreadsheets/d/1EJ8kbB89SYv5fYeeCgds5ue7p_3nyckyN_yujUquJeo/edit#gid=0",
    'MinMaxDeals': "https://docs.google.com/spreadsheets/d/151_G7FTLfJP7y9kypBEF4sYN7gICIrYT/edit",
    'KNTradingLLC': "https://06315f73-014b-478e-b312-e53df7038d3a.filesusr.com/ugd/cc4fa6_9d8a073014d4499d8d117b6724916ea8.xlsx?dn=3_23%20KN%20INVENTORY.xlsx",
    'EcomWholesaleDeals': '1EIzkO1ES0v1VcabxOUQ7VaREIjMnxSgB'
}


In [88]:
url=links['EcomWholesaleDeals']
url

'1EIzkO1ES0v1VcabxOUQ7VaREIjMnxSgB'

In [36]:
df1=get_catalog_google_sheets(url).reset_index(drop=True)
print(df1.shape)
# df1=df1.iloc[9:]
df1.head()


(0, 0)


""


In [45]:
df2=get_catalog_google_sheets_2(url, sheet_name='Sheet1')
df2=make_first_row_header(df2)

df2=df2[[x for x in df2.columns.tolist() if x!=None]]
# df2=df2.dropna()

print(df2.shape)
df2.head()

In [21]:
df3=get_catalog_google_sheets_3(url)
df3.shape


(0, 0)

In [43]:
df4=get_catalog_google_sheets_4(url)
print(df4.shape)
df4.head()

Invalid URL '1EIzkO1ES0v1VcabxOUQ7VaREIjMnxSgB': No scheme supplied. Perhaps you meant http://1EIzkO1ES0v1VcabxOUQ7VaREIjMnxSgB?
(0, 0)


""
